In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

import httpx

# Create HTTP client with verify=False for LangChain
client = httpx.Client(verify=False)

In [10]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
    verify_ssl=False
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

c:\Program Files\Python312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lilianweng.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Total characters: 43047


In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


In [12]:
#Required to avoid SSL verification issues with requests in LangChain/creating embeddings/vector stores
import requests
for method in ("get", "post", "put", "delete", "head", "options", "patch"):
    original = getattr(requests, method)

    def insecure_request(*args, _original=original, **kwargs):
        kwargs["verify"] = False
        return _original(*args, **kwargs)

    setattr(requests, method, insecure_request)


def get_llm():
    llm = ChatOpenAI(
    base_url="https://genailab.tcs.in",
    model="azure_ai/genailab-maas-DeepSeek-V3-0324",
    api_key="sk-xA46m6ExMaLX4lRBXz5gfg",
    http_client=client,
    )

    return llm


def get_embeddings():
    return OpenAIEmbeddings(
        base_url="https://genailab.tcs.in",
        model="azure/genailab-maas-text-embedding-3-large",
        api_key="sk-xA46m6ExMaLX4lRBXz5gfg",
        http_client=client,
        )


def get_vector_store(docs):
    embeddings = get_embeddings()
    # vector_store = FAISS.from_documents(documents=docs, embedding=embeddings)
    # vector_store.save_local('faiss_index')
    vector_store = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="./chroma_index")
    vector_store.persist()
    return vector_store


vector_store = get_vector_store(all_splits)

In [13]:

from langchain.tools import tool

# @tool(response_format="content_and_artifact")

@tool
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


In [14]:
from langchain.agents import create_agent

model = get_llm()

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [15]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

# final_ai_message = None

# for event in agent.stream(
#     {"messages": [{"role": "user", "content": query}]},
#     stream_mode="values",
# ):
#     for msg in event.get("messages", []):
#         if msg.type == "ai":
#             final_ai_message = msg

# # Print only the final AI message
# if final_ai_message:
#     print(final_ai_message.content)




================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_RtT12kBeTtifRRb5IFRo7w)
 Call ID: call_RtT12kBeTtifRRb5IFRo7w
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

('Source: {\'start_index\': 2578, \'source\': \'https://lilianweng.github.io/posts/2023-06-23-agent/\'}\nContent: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\nAnother quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an exter